In [1]:
import matplotlib as plt2 #need this for patches for shaded circles
import numpy as np
import matplotlib.pyplot as plt
import gizmo_analysis as gizmo 
import utilities as ut
import matplotlib.colors as colors
from matplotlib import rc #to use Latex math symbols like 'phi'
import astropy
from astropy.io import ascii
import matplotlib

import pdb
from importlib import reload
# from sl_utilities import distance_functions
#pdb.set_trace()  #<--in case need to troubleshoot
import pickle
import os
from fof_analysis import fof

In [2]:
#########################################################################
#########################################################################
#load particle data here as you normally do to get the star information
#cut on Rxy, |z| and age
#run fof
#sind, sxcm, sycm, szcm, smtot, sgrpid, sr90, sr50, srmax =fof.find(x[si],y[si],z[si], b=b_kpc, mass=mass[si], ncut=ncut_min)
#srcm = np.sqrt(sxcm**2. + sycm**2.)

simname = 'm12i_res7100_mhdcv'
simtype="fire2"
simdir='/scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/'
#simdir = '/scratch/projects/xsede/GalaxiesOnFIRE/cr_suite/m12i_r7100/mhdcv/1Myr/fire2/'
snapnumber = 600
part = gizmo.io.Read.read_snapshots(['star'],'index', snapnumber, simulation_name=simname, simulation_directory=simdir, assign_hosts_rotation=True, assign_hosts=True)  



# in utilities.simulation.Snapshot():
* reading:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/snapshot_times.txt

  using snapshot index = 600, redshift = 0.000


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/output/snapdir_600/snapshot_600.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 57060074 particles
    star      (id = 4): 13976485 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star']

* reading particles from:
    snapshot_600.0.hdf5
    snapshot_600.1.hdf5
    snapshot_600.2.hdf5
    snapshot_600.3.hdf5

* reading cosmological parameters from:  scratch/projects/xsede/GalaxiesOnFIRE/metal_diffusion/m12i_r7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.

In [3]:
rxyz     = part['star'].prop('host.distance.total')
Rxy      = part['star'].prop('host.distance.principal.cylindrical')[:,0]
x        = part['star'].prop('host.distance.principal')[:,0]
y        = part['star'].prop('host.distance.principal')[:,1]
z        = part['star'].prop('host.distance.principal')[:,2] 
age      = part['star'].prop('age')
mass     = part['star'].prop('mass')
ids      = part['star'].prop('id')
id_child = part['star'].prop('id.child')
feh      = part['star'].prop('metallicity.fe')


linking_length = 0.004 #4 parsec (unit here is in kpc)
ncut           = 5 #4 star particles

#keep = np.where((age <= .003) & ((Rxy < 20) & (Rxy>2)) & (abs(z) < 1.5))
#keep = np.where((age <= .003) & (rxyz < 20) & (abs(z) < 1.5))
min_age_value = 5.42
max_age_value = 5.46
keep = np.where((age >= min_age_value) & (age <= max_age_value) & ((Rxy < 20) & (Rxy>2)) & (abs(z) < 1.5))

rxyz0     = rxyz[keep]
Rxy0      = Rxy[keep]
x0        = x[keep]
y0        = y[keep]
z0        = z[keep]
mass0     = mass[keep]
id0       = ids[keep]
id_child0 = id_child[keep]
feh0      = feh[keep]


ind, xcm, ycm, zcm, mtot, grpid, r90, r50, rmax =fof.find(x0,y0,z0, b=linking_length, mass=mass0, ncut=ncut)

linking length  0.004
min number particles in group  5
7.072356224060059 seconds
number of groups 1
max number in a group 5


In [4]:
ngroup = len(mtot)
export_cluster={}
for grp_index in range(ngroup):  #iterate over each group
    cluster={}
    ids_in_cluster = id0[ind[grp_index]]  #these are the star particle ids in each cluster
    id_children_in_cluster = id_child0[ind[grp_index]]
    nstar = len(ids_in_cluster)
    groupid=grpid[grp_index]
    print('------------------------------------------------------------------------------------------------------------------')
    print('grpid, nstar, xcm (kpc), ycm (kpc), zcm (kpc), mtot (msun), rmax (pc)')
    print('%s     %i     %.4f     %.4f    %.4f     %.2e     %.1f ' % (grpid[grp_index], nstar, xcm[grp_index], ycm[grp_index], zcm[grp_index], mtot[grp_index], 1000*rmax[grp_index]))
    print("age os stars in the cluster",age)
    print('ids')
    string = '[' 
    for i in ids_in_cluster:
        string = string + str(i) + ', '

    #get rid of last extra ,
    length = len(string)-2
    string = string[0:length] + ']'
    print(string)

    print('id children')
    string = '[' 
    for i in id_children_in_cluster:
        string = string + str(i) + ', '

    #get rid of last extra ,
    length = len(string)-2
    string = string[0:length] + ']'
    print(string)
    print("These are the ids printed",ids_in_cluster)
    feh_in_cluster=feh0[ind[grp_index]]
    cluster={"cluster_groupid":groupid,"no_of_star":nstar,"id":ids_in_cluster,
    "id_children":id_children_in_cluster,"xcm":xcm[grp_index],"ycm":ycm[grp_index],
    "zcm":zcm[grp_index],"mtot":mtot[grp_index],"r90":r90[grp_index],"r50":r50[grp_index],
    "rmax":rmax[grp_index],"x":x0[ind[grp_index]],"y":y0[ind[grp_index]],"z":z0[ind[grp_index]],"feh":feh_in_cluster}
    export_cluster.update({groupid:cluster})



------------------------------------------------------------------------------------------------------------------
grpid, nstar, xcm (kpc), ycm (kpc), zcm (kpc), mtot (msun), rmax (pc)
1     5     -3.5964     2.4800    -0.9374     4.80e+04     3.5 
age os stars in the cluster [ 9.76040017  1.56987583 11.52149151 ... 12.08954577 11.67528356
 11.67533884]
ids
[19282605, 37408125, 30082663, 38998283, 10956972]
id children
[0, 29, 0, 0, 0]
These are the ids printed [19282605 37408125 30082663 38998283 10956972]


In [5]:
print(export_cluster)

{1: {'cluster_groupid': 1, 'no_of_star': 5, 'id': array([19282605, 37408125, 30082663, 38998283, 10956972], dtype=uint32), 'id_children': array([ 0, 29,  0,  0,  0], dtype=uint32), 'xcm': -3.5963564855064325, 'ycm': 2.4800067280945903, 'zcm': -0.9373722607820829, 'mtot': 48031.23779296875, 'r90': 0.002372313036747412, 'r50': 0.002235945037752304, 'rmax': 0.003531726547188485, 'x': array([-3.5977977 , -3.59806283, -3.59620019, -3.59551358, -3.59360648]), 'y': array([2.48183714, 2.48046878, 2.47866463, 2.4799973 , 2.47931488]), 'z': array([-0.93781988, -0.93795365, -0.93715236, -0.9353013 , -0.93947746]), 'feh': array([ 0.22366543,  0.01020421, -0.10092573, -0.08314352, -0.18668084],
      dtype=float32)}}
